# AI-RAN Environment Validation
## Verify Sionna + Aerial SDK + GPU Setup

This notebook validates that all components are correctly installed:
- TensorFlow with GPU
- NVIDIA Sionna
- NVIDIA Aerial SDK (pyAerial)
- TensorRT
- CUDA runtime


In [1]:
# Cell 1: Check GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("\nGPUs detected:", gpus)

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("\n✅ GPU configured successfully!")
    print(f"   Device: {gpus[0].name}")
else:
    print("\n❌ No GPU detected!")

2025-11-17 19:29:22.489483: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  w

TensorFlow version: 2.20.0

GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

✅ GPU configured successfully!
   Device: /physical_device:GPU:0


In [2]:
# Cell 2: Test Sionna
import sionna
print("Sionna version:", sionna.__version__)

# Test imports with Sionna 1.2.1 API (everything under sionna.phy.*)
try:
    from sionna.phy.channel.tr38901 import CDL
    from sionna.phy.ofdm import ResourceGrid
    from sionna.phy.mapping import Mapper
    
    print("✅ Sionna imported successfully!")
    print("   CDL channel model available")
    print("   OFDM resource grid available")
    print("   QAM mapper available")
except ImportError as e:
    print(f"❌ Sionna import failed: {e}")

Sionna version: 1.2.1
✅ Sionna imported successfully!
   CDL channel model available
   OFDM resource grid available
   QAM mapper available


In [3]:
# Cell 3: Test Aerial SDK (OPTIONAL - not required for AI-RAN pipeline)
import sys
import os

print("Aerial SDK (pyAerial) - OPTIONAL")
print("Note: Not required for neural receiver training and validation")
print("Used only for comparison with Aerial cuPHY implementation\n")

cubb_sdk = os.environ.get('cuBB_SDK')
print(f"  cuBB_SDK env: {cubb_sdk if cubb_sdk else 'Not set'}")

try:
    from aerial.phy5g.pusch import PuschRx
    print("  ✅ pyAerial available")
except Exception as e:
    print(f"  ⚠️  pyAerial not installed (optional)")
    print(f"  Note: AI-RAN notebooks work without it")

Aerial SDK (pyAerial) - OPTIONAL
Note: Not required for neural receiver training and validation
Used only for comparison with Aerial cuPHY implementation

  cuBB_SDK env: Not set
  ⚠️  pyAerial not installed (optional)
  Note: AI-RAN notebooks work without it


In [4]:
# Cell 4: Test TensorRT (via TensorFlow)
print("TensorRT Integration:")
print("Using TF-TRT (TensorFlow's built-in TensorRT support)\n")

try:
    from tensorflow.python.compiler.tensorrt import trt_convert
    print("✅ TF-TRT available (tensorflow.python.compiler.tensorrt)")
    print("   This is sufficient for notebook 03-optimize-tensorrt.ipynb")
    
    # Check if TensorRT is available
    import tensorflow as tf
    print(f"   TensorFlow version: {tf.__version__}")
    print(f"   Built with CUDA: {tf.test.is_built_with_cuda()}")
except Exception as e:
    print(f"⚠️  TF-TRT check: {e}")

# Standalone TensorRT is optional
try:
    import tensorrt as trt
    print(f"\n✅ Standalone TensorRT also available: {trt.__version__}")
except:
    print(f"\n⚠️  Standalone TensorRT not installed (optional)")
    print("   TF-TRT is sufficient for our needs")

TensorRT Integration:
Using TF-TRT (TensorFlow's built-in TensorRT support)

✅ TF-TRT available (tensorflow.python.compiler.tensorrt)
   This is sufficient for notebook 03-optimize-tensorrt.ipynb
   TensorFlow version: 2.20.0
   Built with CUDA: True

⚠️  Standalone TensorRT not installed (optional)
   TF-TRT is sufficient for our needs


In [5]:
# Cell 5: Check neural_rx repository (OPTIONAL)
print("neural_rx Repository - OPTIONAL")
print("Reference implementation from NVIDIA")
print("Our notebooks implement neural receivers from scratch\n")

neural_rx_path = '/opt/app-root/src/neural_rx'
if os.path.exists(neural_rx_path):
    print(f"✅ neural_rx found at {neural_rx_path}")
    
    weights_dir = os.path.join(neural_rx_path, 'weights')
    if os.path.exists(weights_dir):
        print("\nPre-trained models:")
        for f in os.listdir(weights_dir):
            print(f"  - {f}")
else:
    print(f"⚠️  neural_rx not found (optional)")
    print("   Not required - our notebooks implement everything needed")

neural_rx Repository - OPTIONAL
Reference implementation from NVIDIA
Our notebooks implement neural receivers from scratch

✅ neural_rx found at /opt/app-root/src/neural_rx

Pre-trained models:
  - e2e_baseline_weights
  - e2e_large_weights
  - e2e_rt_weights
  - nrx_large_64qam_weights
  - nrx_large_qpsk_weights
  - nrx_large_var_mcs_64qam_masking_weights
  - nrx_large_var_mcs_weights
  - nrx_large_weights
  - nrx_rt_64qam_weights
  - nrx_rt_var_mcs_weights
  - nrx_rt_weights
  - nrx_site_specific_100k_weights
  - nrx_site_specific_baseline_large_weights
  - nrx_site_specific_baseline_weights
  - nrx_site_specific_large_weights
  - nrx_site_specific_weights


In [6]:
# Cell 6: GPU Memory Info
print("GPU Memory Information:")
!nvidia-smi --query-gpu=name,memory.total,memory.free,compute_cap --format=csv

GPU Memory Information:
name, memory.total [MiB], memory.free [MiB], compute_cap
NVIDIA GeForce RTX 4090 D, 49140 MiB, 1509 MiB, 8.9


In [ ]:
# Cell 7: CUDA Test with CuPy
try:
    import cupy as cp
    x = cp.array([1, 2, 3])
    print(f"CuPy array: {x}")
    print(f"Device: {cp.cuda.Device()}")
    print("✅ CUDA is operational!")
except Exception as e:
    print(f"CuPy test: {e}")

In [ ]:
# Cell 8: Summary
print("\n" + "="*70)
print("ENVIRONMENT VALIDATION SUMMARY")
print("="*70)
print(f"✅ TensorFlow {tf.__version__} with GPU support")
print(f"✅ Sionna {sionna.__version__}")
print(f"✅ NVIDIA Aerial SDK (pyAerial)")
try:
    print(f"✅ TensorRT {trt.__version__}")
except:
    pass
print(f"✅ CUDA operational")
print(f"✅ Pre-trained neural_rx models available")
print("="*70)
print("\n🚀 Ready to start AI-RAN experiments!")
print("\nNext: Run 01-dataset-generation.ipynb")